In [29]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install ucimlrepo

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# Dataset

1. load csv file (panda, numpy)

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ucimlrepo import fetch_ucirepo

dataset = fetch_ucirepo(id=94)

x = dataset.data.features
y = dataset.data.targets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

probSpam = y_train[y_train == 1].shape[0] / len(y_train)
probNotSpam = y_train[y_train == 0].shape[0] / len(y_train)

# Naive Bayes Implementation

In [53]:
def naiveBayes(x_train, x_test, y_train, y_test):
  for label in x_train.columns:
    labelProbSpam, labelProbNotSpam = calculate_likelihood(label, x_train, y_train)
    likelihoods[label] = {"spam": labelProbSpam, "notSpam": labelProbNotSpam}

def calculate_likelihood(label, x_train, y_train):
    """Calculate likelihood probabilities."""
    smoothing = 1
    numLabels = x_train.shape[1]
    labelSpamCount = 0
    labelSpamNotCount = 0

    spamTotal = y_train[y_train == 1].shape[0]
    notSpamTotal = y_train[y_train == 0].shape[0]

    for item in range(0, len(x_train[label])):
        if x_train[label].iloc[item] > 0:
          if y_train.iloc[item][0] == 1:
            labelSpamCount += 1
          else:
            labelSpamNotCount += 1

    labelSpam = labelSpamCount / spamTotal
    labelNotSpam = labelSpamNotCount / notSpamTotal

    spamProb = (labelSpam + 1) / (spamTotal + smoothing * numLabels)
    notSpamProb = (labelNotSpam + 1) / (spamTotal + smoothing * numLabels)

    return spamProb, notSpamProb

def naiveBayesClassifier(x_test, likelihoods, spamProb, notSpamProb):
  result = []
  spamLogProb = np.log(spamProb)
  notSpamLogProb = np.log(notSpamProb)

  for row in range(0, len(x_test)):
    for label in x_test.columns:
      if x_test[label].iloc[row] > 0:
        spamLogProb += np.log(likelihoods[label]["spam"])
        notSpamLogProb += np.log(likelihoods[label]["notSpam"])

    if spamLogProb > notSpamLogProb:
      result.append(1)
    else:
      result.append(0)

  return result

#Naive bayes

1. model learning:

   Note:

   features: remove attributes that is not related to word (the last four attributes)

   labels: the last column

   count P(c) -> how many samples are positive, and how many are negtive

   if freq_word>0, then this word exists. You could use this to calculate P(a|c) -> for each class, what is the prob of each word

   remember to use laplace smoothing.

2. model evaluation (on val dataset -> performance(model, val)):
   
   for each new sample, $\prod{P(a|c)}P(c)$; and find the maximum class
   

   

# KNN
1. model learning: None

2. model evaluation(on val dataset): You could use each row(exclude the last column) as the feature of the email. You do not have to recalcuate the freqency.

   ```
   Note:
   parallel programing
   numpy.cos() to calcuate the similarity
   ```

In [81]:
def euclidean_distance(test_row, x_train):
    pointSum = 0
    neighbors = []
    for email in range(0, len(x_train)):
      for label in range(0, len(x_train.columns)):
        pointSum += (test_row.iloc[label] - x_train.iloc[label][email]) ** 2
      neighbors.append(np.sqrt(pointSum))
    return neighbors

def predict(x_train, y_train, x_test, k=3):
    y_pred = []
    for test_point in x_test.iterrows():
        distances = euclidean_distance(test_point, x_train)
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = [y_train.iloc[0][i] for i in k_indices]
        most_common = np.bincount(k_nearest_labels).argmax()
        y_pred.append(most_common)
    return y_pred

# LR

1. model learning: You could use each row(exclude the last column) as the feature of the email. You do not have to recalcuate the freqency.
    
    $y = sigmoid(MX)$

step 1: add one more column (all value is 1) in X -> X' = np.c_[np.ones((len(X), 1)), X]

step 2:vector M = np.random.randn(len(X[0])+1, 1);

key formula for step 3 (Note: n is the size of the TRAINING dataset; $cdot$ is dot production ):

1. $pred_y = sigmoid(M\cdot X')$

2. $loss = -\sum(y\cdot log(pred_y)+(1-y)\cdot log(1-pred_y))/n$

3. $gm=X'\cdot (pred_y - y)*2/n$

Step 3 example code:
   ```
   #Step 3: performing gradient descent on whole dataset:
   best_model = M
   best_performace = 0
   for i in range(epoch):
     pred_y = ...
     gm = ...
     _p = performace(model, val)
     if _p > best_performance:
        best_model = M
        best_performance = _p
     M = M - learning_rate*gm
   ```

2. model evaluation(on val dataset):
  
   calculate pred_y, if more than 0.5, then the predicted label is 1.

In [56]:
def sigmoid(z):
    """Sigmoid function."""
    return 1 / (1 + np.exp(-z))

def cost_function(x, y, theta):
    """Cost function."""
    m = len(y)
    h = sigmoid(np.dot(x, theta))
    return (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))

def gradient_descent(x, y, theta, learning_rate, num_iterations):
    """Gradient descent to optimize parameters."""
    m = len(y)
    costs = []
    for _ in range(num_iterations):
        h = sigmoid(np.dot(x, theta))
        gradient = np.dot(x.T, (h - y)) / m
        theta -= learning_rate * gradient
        cost = cost_function(x, y, theta)
        costs.append(cost)
    return theta, costs

def logistic_regression_train(x_train, y_train, learning_rate=0.01, num_iterations=1000):
    """Train Logistic Regression classifier."""
    intercept = np.ones((x_train.shape[0], 1))
    x_train = np.concatenate((intercept, x_train), axis=1)
    theta = np.zeros(x_train.shape[1])
    theta, _ = gradient_descent(x_train, y_train, theta, learning_rate, num_iterations)
    return theta

def logistic_regression_predict(x_test, theta):
    """Predict using Logistic Regression classifier."""
    intercept = np.ones((x_test.shape[0], 1))
    x_test = np.concatenate((intercept, x_test), axis=1)
    return sigmoid(np.dot(x_test, theta))

# Model Evaluation

https://scikit-learn.org/stable/modules/model_evaluation.html

In [92]:
def assessment(results, y_train):
  falsePos = 0;
  for value in range(0, len(y_train)):
    if results[value] == 1 and y_train["Class"].iloc[value] != 1:
      falsePos += 1
  return falsePos, len(y_train) - falsePos

likelihoods = {}
naiveBayes(x_train, x_test, y_train, y_test)
predictions = naiveBayesClassifier(x_test, likelihoods, probSpam, probNotSpam)
falsePos, truePos = assessment(predictions, y_test)
accuracy = accuracy_score(predictions, y_test)
print("Naive Bayes Accuracy:", accuracy)
print("Naive Bayes False Positives", falsePos)
print("Naive Bayes True Positives", truePos)
'''
k = 5
y_pred = knn_predict(x_train, y_train, x_test, k)
accuracy = accuracy_score(y_test, predictions)
print("KNN Accuracy:", accuracy)
'''
theta = logistic_regression_train(x_train, y_train)
predictions = logistic_regression_predict(x_test, theta)
accuracy = accuracy_score(y_test, predictions)
print("Logistic Regression Accuracy:", accuracy)


Naive Bayes Accuracy: 0.5841476655808904
Naive Bayes False Positives 2
Naive Bayes True Positives 919


ValueError: Unable to coerce to Series, length must be 1: given 3680